In [4]:
import numpy as np
import pandas as pd
import scipy.stats

pd.options.mode.chained_assignment = None
from matplotlib import pyplot as plt, rcParams
# import cv2
import seaborn as sns

sns.set(style="white", context="paper")
from cycler import cycler
import os, sys
import glob
from datetime import datetime, timedelta
from itertools import combinations, product
import base64
from PIL import Image
from io import BytesIO as _BytesIO
import requests
import json
import pickle
from datetime import datetime
from IPython.display import display, Markdown, Latex
from sklearn.metrics import *
import collections
from copy import deepcopy
import traceback
from sympy import Point, Polygon
from decorators import *
from smartprint import smartprint as sprint
from scipy.spatial.distance import cdist
from sklearn.cluster import DBSCAN
# import plotly
# from pandas_profiling import ProfileReport

pd.options.display.max_columns = None
def printm(s): return display(Markdown(s))
    
SERVER_CACHE_DIR = '/mnt/ci-nas-cache/edulyzeV2/cache_compute_4/fixed_face'
os.makedirs(SERVER_CACHE_DIR,exist_ok=True)

track_analysis_meta_cache = f'{SERVER_CACHE_DIR}/analysis_tracking/meta_info'
base_dir = '/mnt/ci-nas-cache/edulyzeV2/pose_face_gaze_emb_fixed_face/'

track_analysis_session_data = f'{SERVER_CACHE_DIR}/analysis_tracking/session_tracking_info'
os.makedirs(track_analysis_session_data,exist_ok=True)

postprocessed_id_map_data_dir = f'{SERVER_CACHE_DIR}/analysis_tracking/processed_id_maps'
os.makedirs(postprocessed_id_map_data_dir, exist_ok=True)

emb_analysis_session_data = f'{SERVER_CACHE_DIR}/analysis_emb/session_emb_info_new'
os.makedirs(emb_analysis_session_data,exist_ok=True)

embmatched_id_raw_data_dir = f'{SERVER_CACHE_DIR}/analysis_emb/embmatched_id_raw'
os.makedirs(embmatched_id_raw_data_dir,exist_ok=True)

embmatched_id_map_data_dir = f'{SERVER_CACHE_DIR}/analysis_tracking/embmatched_id_maps_new'
os.makedirs(embmatched_id_map_data_dir, exist_ok=True)

cross_session_input_data_dir = f'{SERVER_CACHE_DIR}/analysis_emb/cross_session_input'
os.makedirs(cross_session_input_data_dir, exist_ok=True)

id_viz_cache_root = f'{SERVER_CACHE_DIR}/analysis_emb/session_matching_info'
os.makedirs(id_viz_cache_root, exist_ok=True)

## Get frame file data for all sessions

In [5]:

frame_file_data = {}
for course_idx, course_dir in enumerate(glob.glob(f"{base_dir}/*")):
    course_name = course_dir.split("/")[-1]
    course_cache_file = f"{track_analysis_meta_cache}/{course_name}"
    if os.path.exists(course_cache_file):
        frame_file_data[course_name] = pickle.load(open(course_cache_file,"rb"))
        continue
    frame_file_data[course_name]={}
        
    for session_idx, session_dir in enumerate(glob.glob(f"{course_dir}/*")):
        session_name = session_dir.split("/")[-1]
        frame_file_data[course_name][session_name] = {}
        frame_files = glob.glob(f"{session_dir}/*")
        frame_file_names = [xr.split("/")[-1] for xr in frame_files]
        if 'end.pb' in frame_file_names:
            frame_file_data[course_name][session_name]['is_completed']=True
        else:
            frame_file_data[course_name][session_name]['is_completed']=False            
        frame_ids = [int(xr.split(".")[0]) for xr in frame_file_names if not (xr=='end.pb')]
        frame_file_data[course_name][session_name]['frame_ids'] = sorted(frame_ids)
        frame_file_data[course_name][session_name]['dir_location'] = session_dir
        print(f"Got metadata for course: {course_idx}-{course_name}, session:{session_idx}-{session_name}")
    pickle.dump(frame_file_data[course_name],open(course_cache_file,"wb")) 
        
frame_file_data.keys()

dict_keys(['05748A', '05410B', '17214C', '17214B', '05681A', '24352A', '24352C', '86375A'])

# Build final loop for single session compilation

In [6]:
session_filter_list = [
 #    'classinsight-cmu_05681A_ghc_4301_201905011630',
 # 'classinsight-cmu_05681A_ghc_4301_201904171630',
 # 'classinsight-cmu_05681A_ghc_4301_201902201630',
 # 'classinsight-cmu_05681A_ghc_4301_201904101630',
 # 'classinsight-cmu_05681A_ghc_4301_201901231630',
 # 'classinsight-cmu_05418A_ghc_4102_201902251200',
 # 'classinsight-cmu_05418A_ghc_4102_201904081200',
 # 'classinsight-cmu_05418A_ghc_4102_201905011200',
 # 'classinsight-cmu_05418A_ghc_4102_201904291200',
 # 'classinsight-cmu_05418A_ghc_4102_201904011200',
 'classinsight-cmu_05748A_ghc_4101_201902141630',
 'classinsight-cmu_05748A_ghc_4101_201904021630',
 'classinsight-cmu_05748A_ghc_4101_201902051630',
 'classinsight-cmu_05748A_ghc_4101_201902281630',
 'classinsight-cmu_05748A_ghc_4101_201903071630',
 # 'classinsight-cmu_21127J_ghc_4102_201904230930',
 # 'classinsight-cmu_21127J_ghc_4102_201903260930',
 # 'classinsight-cmu_21127J_ghc_4102_201904160930',
 # 'classinsight-cmu_21127J_ghc_4102_201904300930',
 # 'classinsight-cmu_21127J_ghc_4102_201903190930',
 # 'classinsight-cmu_05410A_ghc_4301_201904151500',
 # 'classinsight-cmu_05410A_ghc_4301_201902251500',
 # 'classinsight-cmu_05410A_ghc_4301_201904081500',
 # 'classinsight-cmu_05410A_ghc_4301_201904221500',
 # 'classinsight-cmu_05410A_ghc_4301_201902181500',
                       
 'classinsight-cmu_17214B_ph_a21_201902271030',
 'classinsight-cmu_17214B_ph_a21_201903061030',
 'classinsight-cmu_17214B_ph_a21_201904031030',
 'classinsight-cmu_17214B_ph_a21_201904101030',
 'classinsight-cmu_17214B_ph_a21_201904241030',
 'classinsight-cmu_17214C_ph_225b_201903201130',
 'classinsight-cmu_17214C_ph_225b_201904101130',
 'classinsight-cmu_17214C_ph_225b_201904171130',
 'classinsight-cmu_17214C_ph_225b_201904241130',
 'classinsight-cmu_17214C_ph_225b_201905011130',
 # 'classinsight-cmu_05410B_ghc_4211_201902111500',
 # 'classinsight-cmu_05410B_ghc_4211_201903181500',
 # 'classinsight-cmu_05410B_ghc_4211_201904081500',
 # 'classinsight-cmu_05410B_ghc_4211_201904151500',
 # 'classinsight-cmu_05410B_ghc_4211_201904221500',
 # 'classinsight-cmu_05410B_ghc_4211_201901281500'
]

In [8]:
# writing a generic loop to get embedding info from all courses in frame file data
MAX_GAZE_DEVIATION_DEG = 30
MIN_EMBEDDING_FRAMES = 100
CLU_EPS = 0.4
CLU_MIN_PTS = 100
np.random.seed(42)

for course_idx, course in enumerate(frame_file_data):
    for session_idx, session_id in enumerate(frame_file_data[course]):
        if session_id.split("-front")[0] not in session_filter_list:
            print(f"Session {session_id} not in session filter list, skipping...")
            continue
        cross_session_input_cache_file = f"{cross_session_input_data_dir}/{session_id}.pb"
        try:
            if not os.path.exists(cross_session_input_cache_file):
                printm(f"## Building cross session input info for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
                session_emb_info = pickle.load(open(f'{emb_analysis_session_data}/{session_id}.pb','rb'))
                session_id_map = pickle.load(open(f"{postprocessed_id_map_data_dir}/{session_id}.pb","rb"))
                df_session_eligible_pairs = pd.read_csv(f"{embmatched_id_map_data_dir}/{session_id}.csv")
                
                # Correct new ids with dict from eligible pairs
                eligible_id_map_dict = {}
                for id_pair in df_session_eligible_pairs.id_pair.values:
                    (id1, id2) = eval(id_pair)
                    print(id1, id2)
                    if id2 in eligible_id_map_dict:
                        eligible_id_map_dict[id1] = eligible_id_map_dict[id2]
                    else:
                        eligible_id_map_dict[id2] = id1
                
                sprint(eligible_id_map_dict)

                # correct created eligible map for once more
                for key in sorted(list(eligible_id_map_dict.keys())):
                    key_value = eligible_id_map_dict[key]
                    if key_value in eligible_id_map_dict.keys():
                        eligible_id_map_dict[key] = eligible_id_map_dict[key_value]
                sprint("Corrected", eligible_id_map_dict) 

                sprint("replacing ids in old to new id maps")
                for old_id in session_id_map:
                    if session_id_map[old_id] in eligible_id_map_dict:
                        print(f"replacing {old_id}:{session_id_map[old_id]} -->{eligible_id_map_dict[session_id_map[old_id]]}")
                        session_id_map[old_id] = eligible_id_map_dict[session_id_map[old_id]]
                # sprint({kr:old_to_new_id_map[kr] for kr in old_to_new_id_map if (not old_to_new_id_map[kr]==10000)}) 

                # Replace raw ids with mapped ids after postprocessing for both sessions
                session_emb_info = {
                    xr:{
                        session_id_map[yr]:session_emb_info[xr][yr] 
                            for yr in session_emb_info[xr] if not (session_id_map[yr]==10000)} for xr in session_emb_info}

                # arrange info as per tracking id across both sessions
                gaze_info = {}
                emb_info = {}
                bbox_info = {}
                face_info = {}
                for frame_number in session_emb_info:
                    for trackId in session_emb_info[frame_number]:
                        # trackId = trackId_old if (trackId_old not in eligible_id_map_dict.keys()) else eligible_id_map_dict[trackId_old]
                        if trackId not in gaze_info:
                            gaze_info[trackId] = []
                            emb_info[trackId]=[]
                            bbox_info[trackId] = []
                            face_info[trackId] = []
                        # get  gaze info
                        try:
                            id_bbox = session_emb_info[frame_number][trackId]['bbox']
                            bbox_info[trackId].append([frame_number]+list(id_bbox))
                
                            id_face = session_emb_info[frame_number][trackId]['face'][0]
                            face_info[trackId].append([frame_number]+list(id_face))
                            
                            pitch, roll, yaw= session_emb_info[frame_number][trackId]['rvec'][0]
                            pitch, roll, yaw=np.rad2deg(pitch), np.rad2deg(roll), np.rad2deg(yaw)
                            gaze_sx, gaze_sy, gaze_ex, gaze_ey = session_emb_info[frame_number][trackId]['gaze_2d'][0].flatten()
                            gaze_info[trackId].append([frame_number, pitch, roll, yaw, gaze_sx, gaze_sy, gaze_ex, gaze_ey])
                            face_emb = session_emb_info[frame_number][trackId]['face_embedding'].tolist()
                            emb_info[trackId].append([frame_number]+face_emb)
                        except:
                            continue
                
                for id in gaze_info:
                    gaze_info[id] = pd.DataFrame(gaze_info[id], columns=['frame','pitch','roll','yaw','gaze_sx', 'gaze_sy', 'gaze_ex', 'gaze_ey']).set_index('frame')
                    emb_info[id] =pd.DataFrame(emb_info[id], columns=['frame']+np.arange(512).tolist()).set_index('frame')
                    bbox_info[id] = pd.DataFrame(bbox_info[id], columns=['frame']+np.arange(5).tolist()).set_index('frame')
                    face_info[id] = pd.DataFrame(face_info[id], columns=['frame']+np.arange(15).tolist()).set_index('frame')
                
                sprint({xr:(gaze_info[xr].shape[0],emb_info[xr].shape[0], face_info[xr].shape[0], bbox_info[xr].shape[0]) for xr in emb_info})

                # get gaze based embeddings
                gaze_based_embeddings = {}
                for sid in emb_info.keys():
                    #filter correct frames
                    frames = gaze_info[sid][
                        (gaze_info[sid].yaw.abs()<MAX_GAZE_DEVIATION_DEG) & 
                        (gaze_info[sid].pitch.abs()<MAX_GAZE_DEVIATION_DEG) & 
                        (gaze_info[sid].roll.abs()<MAX_GAZE_DEVIATION_DEG)].index.values
                
                    num_frames = len(frames)
                    if (num_frames<MIN_EMBEDDING_FRAMES):
                        sprint(f"Not sufficient frames to match {sid}:{len(frames)}")
                        continue
                    #get id embeddings    
                    median_emb = np.median(emb_info[sid].loc[frames],axis=0)
                    sprint(f"Got gaze embedding for {sid}.")
                    gaze_based_embeddings[sid]=median_emb

                # get cluster based embeddings
                cluster_based_emb = {}
                for sid in emb_info:
                    emb_clu = DBSCAN(min_samples=CLU_MIN_PTS, eps=CLU_EPS)
                    try:
                        if emb_info[sid].shape[0]>25000:
                            emb_clu.fit(emb_info[sid].sample(25000).values)    
                        else:
                            emb_clu.fit(emb_info[sid].values)
                    except:
                        emb_clu=None
                    if (emb_clu is None) or (max(emb_clu.labels_)<0):
                        sprint(f"All frames are outliers, not proceeding with id {sid}")
                        continue
                    best_cluster_id = pd.Series(emb_clu.labels_[emb_clu.labels_>=0]).value_counts().index[0]
                    frames = emb_info[sid].iloc[emb_clu.labels_==best_cluster_id].index.values
                    cluster_based_emb[sid] = np.median(emb_info[sid].loc[frames],axis=0)
                    sprint(f"Got cluster embedding for {sid}")

                # get complete information needed for a given id for cross-session id tracking
                session_id_info = {}
                for sid in sorted(face_info.keys()):
                    sample_df =deepcopy(bbox_info[sid])
                    
                    sample_df['bbox_width'] = sample_df[2]-sample_df[0]
                    sample_df['bbox_height'] = sample_df[3]-sample_df[1]
                    sample_df['bbox_x'] = (sample_df[2]+sample_df[0]) / 2
                    sample_df['bbox_y'] = (sample_df[3]+sample_df[1]) / 2

                    bbox_width_med, bbox_height_med = sample_df['bbox_width'].median(), sample_df['bbox_height'].median()
                    bbox_width_iqd = sample_df['bbox_width'].quantile(0.75) - sample_df['bbox_width'].quantile(0.25)
                    bbox_height_iqd = sample_df['bbox_height'].quantile(0.75) - sample_df['bbox_height'].quantile(0.25)
                    bbox_x_med, bbox_y_med = sample_df['bbox_x'].median(), sample_df['bbox_x'].median()
                    bbox_x_iqd = sample_df['bbox_x'].quantile(0.75) - sample_df['bbox_x'].quantile(0.25)
                    bbox_y_iqd = sample_df['bbox_y'].quantile(0.75) - sample_df['bbox_y'].quantile(0.25)


                
                    sample_df =deepcopy(face_info[sid])
                    sample_df['face_width'] = sample_df[2]-sample_df[0]
                    sample_df['face_height'] = sample_df[3]-sample_df[1]
                    sample_df['face_height'] = sample_df[3]-sample_df[1]
                    sample_df['face_x'] = (sample_df[2]+sample_df[0]) / 2
                    sample_df['face_y'] = (sample_df[3]+sample_df[1]) / 2
                    
                    face_width_med, face_height_med = sample_df['face_width'].median(), sample_df['face_height'].median()
                    face_width_iqd = sample_df['face_width'].quantile(0.75) - sample_df['face_width'].quantile(0.25)
                    face_height_iqd = sample_df['face_height'].quantile(0.75) - sample_df['face_height'].quantile(0.25)
                    
                    face_x_med, face_y_med = sample_df['face_x'].median(), sample_df['face_x'].median()
                    face_x_iqd = sample_df['face_x'].quantile(0.75) - sample_df['face_x'].quantile(0.25)
                    face_y_iqd = sample_df['face_y'].quantile(0.75) - sample_df['face_y'].quantile(0.25)
                
                    session_id_info[sid] = dict(bbox_width_med=bbox_width_med, bbox_height_med=bbox_height_med,bbox_width_iqd=bbox_width_iqd, bbox_height_iqd=bbox_height_iqd,
                                         bbox_x_med=bbox_x_med, bbox_y_med=bbox_y_med, bbox_x_iqd=bbox_x_iqd, bbox_y_iqd=bbox_y_iqd, 
                                         face_width_med=face_width_med, face_height_med=face_height_med, face_width_iqd=face_width_iqd, face_height_iqd=face_height_iqd,
                                         face_x_med=face_x_med, face_y_med=face_y_med, face_x_iqd=face_x_iqd, face_y_iqd=face_y_iqd, 
                                         cluster_emb = cluster_based_emb.get(sid, None), 
                                         gaze_emb=gaze_based_embeddings.get(sid, None))
                
                sprint(sid, face_width_med, face_width_iqd, face_height_med, face_height_iqd, face_x_med, face_x_iqd, face_y_med, face_y_iqd)
                pickle.dump(session_id_info, open(cross_session_input_cache_file,'wb'))
                printm(f"### Got cross session input info for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
            else:
                ...
                printm(f"## FILE EXISTS: cross session input info for session: {course_idx}-{course}, session:{session_idx}-{session_id}")
        except:
            print(f"ERROR: Unable to get cross session input info for: {course_idx}-{course}, session:{session_idx}-{session_id}")
            print(traceback.format_exc())
    

## FILE EXISTS: cross session input info for session: 0-05748A, session:0-classinsight-cmu_05748A_ghc_4101_201902051630-front

## FILE EXISTS: cross session input info for session: 0-05748A, session:1-classinsight-cmu_05748A_ghc_4101_201902141630-front

Session classinsight-cmu_05748A_ghc_4101_201901291630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201901311630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201902071630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201902121630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201902191630-front not in session filter list, skipping...
Session classinsight-cmu_05748A_ghc_4101_201902211630-front not in session filter list, skipping...


## FILE EXISTS: cross session input info for session: 0-05748A, session:8-classinsight-cmu_05748A_ghc_4101_201902281630-front

Session classinsight-cmu_05748A_ghc_4101_201903051630-front not in session filter list, skipping...


...

"Corrected", eligible_id_map_dict : Corrected {}
"replacing ids in old to new id maps" : replacing ids in old to new id maps
Dict: {xr:(gaze_info[xr].shape[0],emb_info[xr].shape[0], face_info[xr].shape[0], bbox_info[xr].shape[0]) for xr in emb_info}
Key: Value


{0: (11963, 11963, 11963, 12144),
 1: (11459, 11459, 11459, 14984),
 2: (170, 170, 170, 815),
 3: (3286, 3286, 3286, 3328),
 4: (14552, 14552, 14552, 14796),
 5: (14608, 14608, 14608, 14626),
 6: (14098, 14098, 14098, 14339),
 7: (13010, 13010, 13010, 13698),
 8: (13160, 13160, 13160, 13194),
 9: (10139, 10139, 10139, 12126),
 10: (980, 980, 980, 3329),
 11: (10274, 10274, 10274, 11084),
 12: (10623, 10623, 10623, 10638),
 13: (11, 11, 11, 920),
 14: (3, 3, 3, 466),
 15: (354, 354, 354, 1726),
 16: (161, 161, 161, 1938),
 17: (230, 230, 230, 443),
 18: (293, 293, 293, 430),
 19: (492, 492, 492, 697),
 20: (748, 748, 748, 2735),
 21: (79, 79, 79, 415)}

f"Got gaze embedding for {sid}." : Got gaze embedding for 0.
f"Got gaze embedding for {sid}." : Got gaze embedding for 1.
f"Not sufficient frames to match {sid}:{len(frames)}" : Not sufficient frames to match 2:5
f"Got gaze embedding for {sid}." : Got gaze embedding for 3.
f"Got gaze embedding for {sid}." : Got gaze embedding for 4.
f"Got gaze embedding for {sid}." : Got gaze embedding for 5.
f"Got gaze embedding for {sid}." : Got gaze embedding for 6.
f"Got gaze embedding for {sid}." : Got gaze embedding for 7.
f"Got gaze embedding for {sid}." : Got gaze embedding for 8.
f"Got gaze embedding for {sid}." : Got gaze embedding for 9.
f"Got gaze embedding for {sid}." : Got gaze embedding for 10.
f"Got gaze embedding for {sid}." : Got gaze embedding for 11.
f"Not sufficient frames to match {sid}:{len(frames)}" : Not sufficient frames to match 12:79
f"Not sufficient frames to match {sid}:{len(frames)}" : Not sufficient frames to match 13:2
f"Not sufficient frames to match {sid}:{len(frames)

### Got cross session input info for session: 3-17214B, session:9-classinsight-cmu_17214B_ph_a21_201904241030-front

Session classinsight-cmu_17214B_ph_a21_201905011030-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201901301630-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201902061630-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201902131633-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201902201630-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201902271630-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201902271718-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201903131630-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201903201630-front not in session filter list, skipping...
Session classinsight-cmu_05681A_ghc_4301_201903271630-front not in session filter list, skipping...
Se